In [ ]:
#pip install pyrealsense2

In [ ]:
#pip install numpy

In [ ]:
#pip install opencv-python

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2

In [2]:
config=rs.config()
pipeline=rs.pipeline()

In [ ]:
# pipeline_wrapper = rs.pipeline_wrapper(pipeline)
# pipeline_profile = config.resolve(pipeline_wrapper)
# device = pipeline_profile.get_device()
# device_product_line = str(device.get_info(rs.camera_info.product_line))

# Raw OpenCV View from Sensors

In [ ]:
#config.enable_stream(rs.stream.depth,640,480,rs.format.z16,30)
config.enable_stream(rs.stream.color,640,480,rs.format.bgr8,30)
#config.enable_stream(rs.stream.infrared,640,480,rs.format.y8,30)

In [ ]:
pipeline.start(config)

In [ ]:
pipeline.start(config)
while True:
    frames=pipeline.wait_for_frames()
    depth=frames.get_depth_frame()
    colored=frames.get_color_frame()
    infared=frames.get_infrared_frame()
    #rs.log_to_file("log")
    depth_image=np.asanyarray(depth.get_data())
    color_image=np.asanyarray(colored.get_data())
    infared_image=np.asanyarray(infared.get_data())
    depth_colormap=cv2.applyColorMap(cv2.convertScaleAbs(depth_image,alpha=0.03),cv2.COLORMAP_HSV)
    #depth_colormap_dim = depth_colormap.shape
    #color_colormap_dim = color_image.shape
    #images = np.hstack((color_image, depth_colormap))
    cv2.namedWindow('Depth View', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Depth View', depth_colormap)
#     cv2.imshow('Infared Feed',infared_image)
#     cv2.imshow('Depth Feed',depth_image)
#     cv2.imshow('Color Feed',color_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
pipeline.stop()
cv2.destroyAllWindows()

# Depth Image

In [3]:
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

In [10]:
print(profile.get_device())

<pyrealsense2.device: Intel RealSense D435 (S/N: 745412071635  FW: 5.15.0.2  on USB3.2)>


In [ ]:
pipeline.stop()

In [4]:
depth_sensor=profile.get_device().first_depth_sensor()
depth_scale=depth_sensor.get_depth_scale()
print("Depth Scale is: ",depth_scale)
clipping_distance = 1 / depth_scale #remove objects after 1m

Depth Scale is:  0.0010000000474974513


In [5]:
align_to = rs.stream.color
align = rs.align(align_to)

In [6]:
while True:
    frames=pipeline.wait_for_frames()
    aligned_frames = align.process(frames)
    aligned_depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()
    depth_image = np.asanyarray(aligned_depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    grey_color = 153
    depth_image_3d = np.dstack((depth_image,depth_image,depth_image))
    bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)
    depth_colormap=cv2.applyColorMap(cv2.convertScaleAbs(depth_image,alpha=0.03),cv2.COLORMAP_HSV)
    images = np.hstack((bg_removed, depth_colormap))
    cv2.namedWindow('Depth View', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Depth View', images)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
pipeline.stop()
cv2.destroyAllWindows()

In [ ]:
pipeline.stop()

In [ ]:
pipeline.start()
try:
    for i in range(0,100):
        frames=pipeline.wait_for_frames()
        depth=frames.get_depth_frame()
        for f in frames:
            print(f.profile)
finally:
    pipeline.stop()